# Project: Image Classification
# By Omar Diab [20p3176]

# ***Downloading Content***

In [ ]:
%matplotlib inline

In [ ]:
from __future__ import print_function, division
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

In [ ]:
! gdown --id 1pL8D87UZ7ioj43ob14pPkCvJbQdTOwme

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1pL8D87UZ7ioj43ob14pPkCvJbQdTOwme
To: /content/ConvNext Tiny
100% 111M/111M [00:00<00:00, 229MB/s] 


In [ ]:
! gdown --id 1fbsnjPsrD2hBWSZdmDmwJpHqsVhQfvhm

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1fbsnjPsrD2hBWSZdmDmwJpHqsVhQfvhm
To: /content/sprints-ai-and-ml-competition-2022-round-2.zip
100% 254M/254M [00:01<00:00, 185MB/s]


In [ ]:
!unzip "/content/sprints-ai-and-ml-competition-2022-round-2.zip" -d "/content/sprints-ai-and-ml-competition-2022-round-2"

Streaming output truncated to the last 5000 lines.
  inflating: /content/sprints-ai-and-ml-competition-2022-round-2/Scenes training set/mountain/7551.jpg  
  inflating: /content/sprints-ai-and-ml-competition-2022-round-2/Scenes training set/mountain/7560.jpg  
  inflating: /content/sprints-ai-and-ml-competition-2022-round-2/Scenes training set/mountain/7565.jpg  
  inflating: /content/sprints-ai-and-ml-competition-2022-round-2/Scenes training set/mountain/7578.jpg  
  inflating: /content/sprints-ai-and-ml-competition-2022-round-2/Scenes training set/mountain/7581.jpg  
  inflating: /content/sprints-ai-and-ml-competition-2022-round-2/Scenes training set/mountain/7586.jpg  
  inflating: /content/sprints-ai-and-ml-competition-2022-round-2/Scenes training set/mountain/7647.jpg  
  inflating: /content/sprints-ai-and-ml-competition-2022-round-2/Scenes training set/mountain/7652.jpg  
  inflating: /content/sprints-ai-and-ml-competition-2022-round-2/Scenes training set/mountain/7654.jpg  
  in

# ***Spliting Data into train and val***

In [ ]:
!pip install split-folders[full]
import splitfolders
# spliting Training to Train and val
split_dir = '/content/sprints-ai-and-ml-competition-2022-round-2/Scenes training set'

splitfolders.ratio(split_dir,output='output',ratio=(.85,.15))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Copying files: 14034 files [00:02, 6803.64 files/s]


# ***Data Augmentation***

In [ ]:


data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((150,150)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'val': transforms.Compose([
        transforms.Resize((150,150)),
        transforms.CenterCrop(150),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
}


data_dir = '/content/output'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=14)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 14 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# ***Function to Train the Model***

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):   # we need to change number of epochs
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model
from socket import socket

In [ ]:
# forcing using gpu
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

# ***ResNet 50***

In [ ]:
model_ft50 = models.resnet50(pretrained=True)
num_ftrs = model_ft50.fc.in_features

model_ft50.fc = nn.Linear(num_ftrs, 6)

model_ft = model_ft50.to(device)

criterion = nn.CrossEntropyLoss()


optimizer_ft = optim.SGD(model_ft50.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
#model_ft50 = train_model(model_ft50, criterion, optimizer_ft, exp_lr_scheduler,
#                   num_epochs=15)

# ***ResNest 18***

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, 6)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()


optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                   num_epochs=15)

# ***ConvNext_Tiny***

In [ ]:
model_conv = torchvision.models.convnext_tiny(pretrained=True)

model_conv.classifier[2]=nn.Linear(model_conv.classifier[2].in_features, 6)


model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()


optimizer_conv = optim.SGD(model_conv.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [ ]:
model_conv = train_model(model_conv, criterion, optimizer_conv,exp_lr_scheduler, num_epochs=17)

Epoch 0/4
----------
train Loss: 0.6010 Acc: 0.8383
val Loss: 0.3031 Acc: 0.9056

Epoch 1/4
----------
train Loss: 0.2719 Acc: 0.9101
val Loss: 0.2385 Acc: 0.9231

Epoch 2/4
----------
train Loss: 0.2269 Acc: 0.9235
val Loss: 0.2188 Acc: 0.9231

Epoch 3/4
----------
train Loss: 0.2035 Acc: 0.9324
val Loss: 0.2311 Acc: 0.9194

Epoch 4/4
----------
train Loss: 0.1889 Acc: 0.9360
val Loss: 0.2050 Acc: 0.9293

Training complete in 8m 59s
Best val Acc: 0.929317


# ***ConVNext_Small***

In [ ]:
model_conv = torchvision.models.convnext_small(pretrained=True)

model_conv.classifier[2]=nn.Linear(model_conv.classifier[2].in_features, 6)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_conv = optim.SGD(model_conv.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ConvNeXt_Small_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/convnext_small-0c510722.pth" to /root/.cache/torch/hub/checkpoints/convnext_small-0c510722.pth


  0%|          | 0.00/192M [00:00<?, ?B/s]

In [ ]:
model_conv = train_model(model_conv, criterion, optimizer_conv,exp_lr_scheduler, num_epochs=17)

Epoch 0/16
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 14 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 0.3447 Acc: 0.8808
val Loss: 0.2642 Acc: 0.9028

Epoch 1/16
----------
train Loss: 0.1993 Acc: 0.9303
val Loss: 0.2059 Acc: 0.9307

Epoch 2/16
----------
train Loss: 0.1728 Acc: 0.9386
val Loss: 0.1996 Acc: 0.9288

Epoch 3/16
----------
train Loss: 0.1585 Acc: 0.9424
val Loss: 0.2025 Acc: 0.9298

Epoch 4/16
----------
train Loss: 0.1409 Acc: 0.9468
val Loss: 0.1852 Acc: 0.9388

Epoch 5/16
----------
train Loss: 0.1325 Acc: 0.9534
val Loss: 0.2078 Acc: 0.9312

Epoch 6/16
----------
train Loss: 0.1227 Acc: 0.9540
val Loss: 0.1959 Acc: 0.9322

Epoch 7/16
----------
train Loss: 0.0935 Acc: 0.9689
val Loss: 0.1814 Acc: 0.9393

Epoch 8/16
----------
train Loss: 0.0859 Acc: 0.9718
val Loss: 0.1829 Acc: 0.9402

Epoch 9/16
----------
train Loss: 0.0805 Acc: 0.9734
val Loss: 0.1843 Acc: 0.9412

Epoch 10/16
----------
train Loss: 0.0791 Acc: 0.9721
val Loss: 0.1842 Acc: 0.9407

Epoch 11/16
----------
train Loss: 0.0777 Acc: 0.9735
val Loss: 0.1877 Acc: 0.9402

Epoch 12/16
----------
t

# ***Saving Model***

In [ ]:
save_dir='/content/Trained_model'
torch.save(model_conv,save_dir) #SAVES THE TRAINED MODEL


# ***Loading Model***

In [ ]:
model=torch.load('/content/ConvNext Tiny')
model.eval()

ConvNeXt(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (1): Permute()
          (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=96, out_features=384, bias=True)
          (4): GELU(approximate=none)
          (5): Linear(in_features=384, out_features=96, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (1): Permute()
          (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=96

# ***Testing Phase***

In [ ]:
from re import X
import PIL.Image as im
#from PIL import Image
import os
from os import listdir
from glob import glob
def predict(model,image_transforms,images_paths,classes):
    list1=[]
    list2=[]
    for i in images_paths:
     print(i)
     model=model.eval()
     image = im.open(i)
     image=image_transforms(image).to('cuda').float()
     image=image.unsqueeze(0)
     output=model(image)
     _, predicted=torch.max(output.data,1)

     print(i[-9:],predicted.item())
     list1.append(i[-9:])
     list1.append(predicted.item())
     list2.append(list1)
     list1=[]
    return list2





In [ ]:
Test_dir='/content/sprints-ai-and-ml-competition-2022-round-2/Scenes testing test'

tranform_test = transforms.Compose([transforms.Resize((150,150)),transforms.CenterCrop(150),
        transforms.ToTensor(), transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
                 ])


In [ ]:
x=os.listdir(Test_dir)

list_path=[]
for i in range(len(x)):
     list_path.append(Test_dir+"/"+str(x[i]))

In [ ]:
data_frame=predict(model,tranform_test,x,class_names)

# ***Importing to CSV***

In [ ]:
import pandas as pd
New_Data_Frame=pd.DataFrame(data_frame)
New_Data_Frame.to_csv('/content/sprints-ai-and-ml-competition-2022-round-2/submit.csv',index=False)
